In [1]:
#!/usr/bin/env python3
#coding: utf-8

#基于训练好的词向量模型进行聚类
#聚类采用Kmeans算法
import math
import re
import json
import jieba
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from collections import defaultdict

In [2]:
#输入模型文件路径
#加载训练好的模型
def load_word2vec_model(path):
    model = Word2Vec.load(path)
    return model

def load_sentence(path):
    sentences = set()
    with open(path, encoding="utf8") as f:
        for line in f:
            sentence = line.strip()
            sentences.add(" ".join(jieba.cut(sentence)))
    print("获取句子数量：", len(sentences))
    return sentences

In [3]:
#将文本向量化
def sentences_to_vectors(sentences, model):
    vectors = []
    for sentence in sentences:
        words = sentence.split()  #sentence是分好词的，空格分开
        vector = np.zeros(model.vector_size)
        #所有词的向量相加求平均，作为句子向量
        for word in words:
            try:
                vector += model.wv[word]
            except KeyError:
                #部分词在训练中未出现，用全0向量代替
                vector += np.zeros(model.vector_size)
        vectors.append(vector / len(words))
    return np.array(vectors)

In [4]:
def main():
    model = load_word2vec_model("model.w2v") #加载词向量模型
    sentences = load_sentence("titles.txt")  #加载所有标题
    vectors = sentences_to_vectors(sentences, model)   #将所有标题向量化

    n_clusters = int(math.sqrt(len(sentences)))  #指定聚类数量
    print("指定聚类数量：", n_clusters)
    kmeans = KMeans(n_clusters)  #定义一个kmeans计算类
    kmeans.fit(vectors)          #进行聚类计算

    sentence_label_dict = defaultdict(list)
    centers = kmeans.cluster_centers_  # 获取聚类中心
    for sentence, vector, label in zip(sentences, vectors, kmeans.labels_):
        obj = {}
        distance = np.linalg.norm(vector - centers[label])  # 计算距离
        obj["sentence"] = sentence
        obj["distance"] = distance
        obj["label"] = label
        sentence_label_dict[label].append(obj)  # 同标签的放到一起
    for label, objs in sentence_label_dict.items():
        sorted_objs = sorted(objs, key=lambda x: x["distance"])  # 按距离排序
        print("cluster %s :" % label)
        for index, vector in enumerate(sorted_objs):
            print("距离 %s : %s" % (vector["distance"], vector["sentence"].replace(" ", "")))
        print("==========================")

In [5]:
if __name__ == "__main__":
    main()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\WUJIAY~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.650 seconds.
Prefix dict has been built successfully.


获取句子数量： 1796
指定聚类数量： 42
cluster 11 :
距离 0.3441780372617346 : 组图：王菲个性高跟鞋尽显巨星范儿
距离 0.4650935842875335 : 组图：女星展示性感身材内衣选择有一套
距离 0.48924962662482874 : 搭配QA：最显瘦高8款韩式长短搭
距离 0.5028757501172069 : 新实穿：春夏必备五类时尚牛仔装备
距离 0.5076596366142736 : 组图：迷你裙搭秋靴美女示范
距离 0.5104158059020013 : 组图：女星小皮衣换季帅气又可爱
距离 0.5122625920518286 : 组图：日系内衣来袭给你始料不及的性感
距离 0.5152993438022787 : 组图：四川妹子的街头麻辣装扮
距离 0.5249352902163334 : 花边报：反面教材明星错穿褶皱时装
距离 0.5322006436270833 : 组图：韩国当红女星优雅知性装
距离 0.5398355896223751 : 组图：女星穿高腰牛仔裤出丑瞬间
距离 0.5448137416043106 : 组图：甜美Girl扮靓一周的小秘诀
距离 0.5528708924102211 : 组图：绚丽甲虫媲美宝石
距离 0.5554957288725502 : 组图：街拍春天里的百姓时尚
距离 0.5669660020731665 : 组图：大胸女跟巩俐学穿衣
距离 0.5763561614496513 : 组图：16款显瘦叠穿造型吸引他
距离 0.5963442020170386 : 搭配QA：初秋购入什么衣服最能体现品位？
距离 0.6110847542505778 : 组图：明星出位街头装抢先看
距离 0.6167584311743518 : 组图：搭配各种冬鞋都OK的5分裤
距离 0.6189996397682059 : 组图：本季最热门塑造时髦造型的经典小马甲
距离 0.6340655293130926 : 组图：最常见三种脸型帽子搭配术
距离 0.6353628254829876 : 组图：12款日本街头妹最常见搭配
距离 0.6397304910800025 : 组图：不同的红色打造神秘女郎
距离 0.6411857825275097 : 搭配QA：如何在冬季